# How to Fine-Tune LLMs with LoRA Adapters using Hugging Face TRL

This notebook demonstrates how to efficiently fine-tune large language models using LoRA (Low-Rank Adaptation) adapters. LoRA is a parameter-efficient fine-tuning technique that:
- Freezes the pre-trained model weights
- Adds small trainable rank decomposition matrices to attention layers
- Typically reduces trainable parameters by ~90%
- Maintains model performance while being memory efficient

We'll cover:
1. Setup development environment and LoRA configuration
2. Create and prepare the dataset for adapter training
3. Fine-tune using `trl` and `SFTTrainer` with LoRA adapters
4. Test the model and merge adapters (optional)


## 1. Setup development environment

Our first step is to install Hugging Face Libraries and Pytorch, including trl, transformers and datasets. If you haven't heard of trl yet, don't worry. It is a new library on top of transformers and datasets, which makes it easier to fine-tune, rlhf, align open LLMs.


In [1]:
# Install the requirements in Google Colab
!pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login

# login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.1/231.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.1/344.1 kB 22.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    F

## 2. Load the dataset

In [5]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
# dataset = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations", split="train")
dataset = load_dataset(path="prithivMLmods/Math-Solve", split="train[:25000]")

def format_dataset(sample):
    return {"content": [sample["problem"], sample["solution"]]}

def format_dataset(sample):
    return {"text": sample["problem"], "label": sample["solution"]}

def format_dataset(sample):
    content = [
        {"role": "system", "content": "You are helpful"},
        {"role": "user", "content": sample["problem"]},
        {"role": "assistant", "content": sample["solution"]}
    ]
    return {"messages":content}

dataset = dataset.map(format_dataset)


dataset = load_dataset(path="Qurtana/medical-o1-reasoning-SFT-orpo",  split="train")


def format(sample):
    content = [
        {"role": "system", "content": "You are helpful assistant"},
        {"role": "user", "content": sample["prompt"]},
        {"role": "assistant", "content": sample["accepted"]}
    ]
    return {"messages":content}

dataset=dataset.map(format)

dataset_train = dataset.select(range(20000))
dataset_test = dataset.select(range(dataset.shape[0]-2000, dataset.shape[0]))

print(dataset)

Map:   0%|          | 0/25371 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'accepted', 'rejected', '_original_idx', 'messages'],
    num_rows: 25371
})


In [6]:
for x in dataset[0]["messages"]:
  print(x)

{'content': 'You are helpful assistant', 'role': 'system'}
{'content': 'A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?', 'role': 'user'}
{'content': "Okay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her abdominal pressure like coughing or sneezing. This sounds a lot like stress urinary incontinence to me. Now, it's interesting that she doesn't have any issues at night; she isn't experiencing leakage while sleeping. This likely means her bladder's ability to hold urine is fine when she isn't under physical stress. Hmm, that's a clue that we're dealing with something related to pressure rather than a bladder muscle problem.

## 3. Fine-tune LLM using `trl` and the `SFTTrainer` with LoRA

The [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer) from `trl` provides integration with LoRA adapters through the [PEFT](https://huggingface.co/docs/peft/en/index) library. Key advantages of this setup include:

1. **Memory Efficiency**:
   - Only adapter parameters are stored in GPU memory
   - Base model weights remain frozen and can be loaded in lower precision
   - Enables fine-tuning of large models on consumer GPUs

2. **Training Features**:
   - Native PEFT/LoRA integration with minimal setup
   - Support for QLoRA (Quantized LoRA) for even better memory efficiency

3. **Adapter Management**:
   - Adapter weight saving during checkpoints
   - Features to merge adapters back into base model

We'll use LoRA in our example, which combines LoRA with 4-bit quantization to further reduce memory usage without sacrificing performance. The setup requires just a few configuration steps:
1. Define the LoRA configuration (rank, alpha, dropout)
2. Create the SFTTrainer with PEFT config
3. Train and save the adapter weights


In [ ]:
# Import necessary libraries
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# device = "cuda:1"


from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

print(torch.cuda.is_available())

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)


# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    device_map = device,
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset-sft-peft"
finetune_tags = ["smol-course", "module_1"]

print(model.device)

True
cuda:0


The `SFTTrainer`  supports a native integration with `peft`, which makes it super easy to efficiently tune LLMs using, e.g. LoRA. We only need to create our `LoraConfig` and provide it to the trainer.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Define LoRA parameters for finetuning</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p>
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the general parameters for an abitrary finetune</p>
    <p>🐕 Adjust the parameters and review in weights & biases.</p>
    <p>🦁 Adjust the parameters and show change in inference results.</p>
</div>

In [ ]:
from peft import LoraConfig

# TODO: Configure LoRA parameters
# r: rank dimension for LoRA update matrices (smaller = more compression)
rank_dimension = 32
# lora_alpha: scaling factor for LoRA layers (higher = stronger adaptation)
lora_alpha = 48
# lora_dropout: dropout probability for LoRA layers (helps prevent overfitting)
lora_dropout = 0.05

peft_config = LoraConfig(
    r=rank_dimension,  # Rank dimension - typically between 4-32
    lora_alpha=lora_alpha,  # LoRA scaling factor - typically 2x rank
    lora_dropout=lora_dropout,  # Dropout probability for LoRA layers
    bias="none",  # Bias type for LoRA. the corresponding biases will be updated during training.
    target_modules="all-linear",  # Which modules to apply LoRA to
    task_type="CAUSAL_LM",  # Task type for model architecture
)


Before we can start our training we need to define the hyperparameters (`TrainingArguments`) we want to use.

In [ ]:
# Training configuration
# Hyperparameters based on QLoRA paper recommendations
max_seq_length = 1512  # max sequence length for model and packing of the dataset

args = SFTConfig(
    # Small learning rate to prevent catastrophic forgetting
    learning_rate=1e-5,
    # Linear learning rate decay over training
    lr_scheduler_type="linear",
    # Maximum combined length of prompt + completion
    max_length=1024,
    # Maximum length for input prompts
    max_prompt_length=512,
    # Batch size for training
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    # Helps with training stability by accumulating gradients before updating
    gradient_accumulation_steps=4,
    # Memory-efficient optimizer for CUDA, falls back to adamw_torch for CPU/MPS
    optim="paged_adamw_8bit" if device == "cuda" else "adamw_torch",
    # Number of training epochs
    num_train_epochs=1,
    # When to run evaluation
    evaluation_strategy="steps",
    # Evaluate every 20% of training
    eval_steps=0.2,
    # Log metrics every step
    logging_steps=1,
    # Gradual learning rate warmup
    warmup_steps=10,
    # Disable external logging
    report_to="wandb",
    # Where to save model/checkpoints
    output_dir=finetune_name,
    # Enable MPS (Metal Performance Shaders) if available
    use_mps_device=device == "mps",
    hub_model_id=finetune_name,
    # Use bfloat16 precision for faster training
    bf16=True,
)

We now have every building block we need to create our `SFTTrainer` to start then training our model.

In [ ]:

max_seq_length = 1512  # max sequence length for model and packing of the dataset

# Create SFTTrainer with LoRA configuration
trainer = SFTTrainer(
    model=model,
    args=args,
    peft_config=peft_config,  # LoRA configuration
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    processing_class=tokenizer,
)



/tmp/ipykernel_298318/678892860.py:4: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


In [ ]:
# print(trainer.train_dataset[0])
# print("\n\n")
# print(tokenizer.decode(trainer.train_dataset[0]["input_ids"]))
# print("\n\n")
# print(tokenizer.decode(trainer.train_dataset[0]["labels"]))


Start training our model by calling the `train()` method on our `Trainer` instance. This will start the training loop and train our model for 3 epochs. Since we are using a PEFT method, we will only save the adapted model weights and not the full model.

In [ ]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save model
trainer.save_model()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: matteob-90-hotmail-it. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,2.027300
20,1.779000
30,1.786700
40,1.762600
50,1.797300
60,1.885600
70,1.837500
80,1.632900
90,1.771900
100,1.643800


KeyboardInterrupt: 

The training with Flash Attention for 3 epochs with a dataset of 15k samples took 4:14:36 on a `g5.2xlarge`. The instance costs `1.21$/h` which brings us to a total cost of only ~`5.3$`.



### Merge LoRA Adapter into the Original Model

When using LoRA, we only train adapter weights while keeping the base model frozen. During training, we save only these lightweight adapter weights (~2-10MB) rather than a full model copy. However, for deployment, you might want to merge the adapters back into the base model for:

1. **Simplified Deployment**: Single model file instead of base model + adapters
2. **Inference Speed**: No adapter computation overhead
3. **Framework Compatibility**: Better compatibility with serving frameworks


In [ ]:
from peft import AutoPeftModelForCausalLM


# Load PEFT model on CPU
model = AutoPeftModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=args.output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# Merge LoRA and base model and save
merged_model = model.merge_and_unload()
merged_model.save_pretrained(
    args.output_dir, safe_serialization=True, max_shard_size="2GB"
)

## 3. Test Model and run Inference

After the training is done we want to test our model. We will load different samples from the original dataset and evaluate the model on those samples, using a simple loop and accuracy as our metric.



<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Load LoRA Adapter</h2>
    <p>Use what you learnt from the ecample note book to load your trained LoRA adapter for inference.</p>
</div>

In [ ]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

# Load Model with PEFT adapter
tokenizer = AutoTokenizer.from_pretrained(finetune_name)
model = AutoPeftModelForCausalLM.from_pretrained(
    finetune_name, device_map="auto", torch_dtype=torch.float16
)
pipe = pipeline(
    "text-generation",
    model=merged_model,
    tokenizer=tokenizer,
    device=device,
    max_new_tokens=200,
    num_beams=10,
    num_return_sequences=1,
    no_repeat_ngram_size=1,
)

Device set to use cuda


Lets test some prompt samples and see how the model performs.

In [ ]:
prompts = [
    "What is the capital of Germany? Explain why thats the case and if it was different in the past?",
    "Write a Python function to calculate the factorial of a number.",
    "A rectangular garden has a length of 25 feet and a width of 15 feet. If you want to build a fence around the entire garden, how many feet of fencing will you need?",
    "What is the difference between a fruit and a vegetable? Give examples of each.",
]


def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        tokenize=False,
        add_generation_prompt=True,
    )
    outputs = pipe(
        prompt,
    )
    return outputs[0]["generated_text"][len(prompt) :].strip()


for prompt in prompts:
    print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(prompt)}")
    print("-" * 50)

    prompt:
What is the capital of Germany? Explain why thats the case and if it was different in the past?
    response:
To answer this question, we need to know more about German history. We can start by looking at some historical figures who have made a significant impact on our country's development:

  • Otto von Bismark (1835 - 90): A Prussian military leader known for his strong leadership during World War II when he served as chancellor under President Franklin D Roosevelt after Adolf Hitler had been re-elected president several times before coming down with an illness called "Mein Kampf" ("My Struggle"). He introduced many important reforms like implementing conscription into young men aged fifteen or sixteen years old so they could serve their countries' needs better than any other nation did back then! His legacy continues today through modern education policies promoting critical thinking skills among future generations while also encouraging civic engagement across various

prompt:
What is the capital of Germany? Explain why thats the case and if it was different in the past?
    response:
To answer this question, we need to look at some historical facts. In 1870 there were two German states: Baden-Wuerttemberg (German for "Bavaria with Wurttenberg" or Bavarian Palatinate), which had a long history as an independent principality before being annexed by Prussia during World War II; Saxony/Saxonschweiz ("Saxon Schleswig"), whose territory came under Prussian control after gaining its independence from Denmark through negotiations between Otto von Bismarck's Chancellor Wilhelmina zu Hohenzollern Wilhelm I & King Frederick William IV on August 26th - September  3rd. The former state has since been reincorporated into Süddeutsche Bundesrepubliken / Deutsches Volkskreuzgefahrzeug [DWDG], also called Deutschland über all diese Zeit ["Germany Under All Time
--------------------------------------------------
    prompt:
Write a Python function to calculate the factorial of a number.
    response:
Let $n$ be an integer greater than 1, and let $\sum_{k=0}^{\infty} n = \frac{3}{2!}$ for some non-zero prime power $(p_i)$. We can rewrite this expression as follows: (Note that we are working with positive integers here.) The sum is computed by adding up each term in its own row or column until it reaches zero; if there were no terms between two consecutive numbers then they would have been added together at least once before being subtracted from their respective rows/colonelstheoreticalexpansionofthefactorialfunctioniscomputablebymultiplying themtogetherandaddingone moreterm). This gives us$\left(\dfrac{\pi^4}{\sqrt[5]{9}}+\cdots+7^{6}\right)$for all primes not exceeding one hundred thousand but excluding those whose only common divisor divides evenly into foursquare root(which means
--------------------------------------------------
    prompt:
A rectangular garden has a length of 25 feet and a width of 15 feet. If you want to build a fence around the entire garden, how many feet of fencing will you need?
    response:
Let's start by thinking about what we are looking for in this problem: How much space is there between two consecutive points on an equilateral triangle (a rectangle)? We know that it can be written as $\triangle ABC$, where $AB = \frac{4}{3} A + B$. So if I add up all three sides along with their corresponding angles ($\overrightarrow {ABC}$$=60^\circ) then my equation becomes $(AD+BC)\cos(\theta)+(AC+\Delta)=897.\]I have already seen some similar problems involving right triangles but they don't work out so well when using area or perimeter formulas! Let me try one more example from real-life situations before moving onto our main task - finding total cost per square foot without considering any extra costs like utility bills etc., which might come into play at later stages depending upon various factors mentioned earlier during your math practice session...[SOLVED](https://www
--------------------------------------------------
    prompt:
What is the difference between a fruit and a vegetable? Give examples of each.
    response:
The word "fruit" refers to something that grows on trees, such as an apple or banana; it can also be used for any edible plant with seeds inside (like tomatoes). Vegetables are different from fruits because they don't produce their own food through photosynthesis but instead grow by eating other living things called plants! Some common vegetables include broccoli florets (*), carrots*, bell peppers**, zucchinis**. You might have seen them growing in your garden when you were younger - just think about all those delicious little green ones popping up every time someone goes outside during hot summer days... That's what we call'vegetables'. Now let me explain how these two categories differ:

1️⃣ **Fruits**: Fruits contain natural sugars like watermelons *and* grapes which give us energy while satisfying our sweet tooth without adding extra calories.* They're usually eaten fresh rather than cooked down into ice cream sundaes at birthday parties.**[Example:* An orange contains 90
--------------------------------------------------